In [19]:
import numpy as np
import pandas as pd
from meteostat import *
from datetime import datetime

In [20]:
Daily.cache_dir = '/media/andrea/EEE438B1E4387E41/machine_learning/meteo'
stations = Stations().region('FR')

In [22]:
start = datetime(2010, 1, 1)
end = datetime(2021, 1, 1)

In [23]:
stations.count()

223

In [24]:

stations.inventory('daily', (start, end))
stations = stations.fetch()


In [25]:
stations

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,,,,,
07002,Boulogne,FR,O,07002,<NA>,50.7333,1.6000,70.0,Europe/Paris,1973-01-01,2023-04-09,1999-03-24,2022-03-28,2005-01-01,2021-01-01
07003,Le Touquet,FR,O,07003,LFAT,50.5167,1.6167,10.0,Europe/Paris,1973-01-01,2023-05-18,2004-10-19,2022-04-25,2005-01-01,2022-01-01
07005,Abbeville,FR,S,07005,LFOI,50.1333,1.8333,74.0,Europe/Paris,1926-10-04,2023-05-18,1940-06-08,2023-05-10,1940-01-01,2022-01-01
07010,Dunkerque,FR,O,07010,<NA>,51.0500,2.3333,6.0,Europe/Paris,1973-01-01,2023-04-09,1999-03-24,2022-03-28,2009-01-01,2021-01-01
07015,Lille,FR,O,07015,LFQQ,50.5667,3.1000,48.0,Europe/Paris,1945-03-01,2023-05-18,1945-03-02,2023-05-10,1973-01-01,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LFRV0,Vannes-Meucon,FR,E,<NA>,LFRV,47.7233,-2.7186,136.0,Europe/Paris,1987-06-01,2023-05-18,2004-10-20,2022-04-25,2008-01-01,2022-01-01
LFSG0,Epinal / Baudricourt,FR,M,<NA>,LFSG,48.3167,6.0667,331.0,Europe/Paris,2007-02-27,2023-05-18,2014-05-29,2022-04-25,2015-01-01,2022-01-01
LFSM0,Montbéliard-Courcelles / Montbeliard-Courcelle...,FR,I,<NA>,LFSM,47.4870,6.7905,317.0,Europe/Paris,2021-01-04,2023-05-17,2021-01-04,2022-04-24,2021-01-01,2022-01-01


In [26]:
stations =  stations[(stations['longitude']  <= 8) & (stations['longitude']  >=  -10)] 

In [ ]:
data = []
true_stations = []

for station in stations.index : 

   df = Daily(station, start, end)
   
   if df.coverage() >=  0.7 :
      
      true_stations.append(station)
      df = df.normalize().interpolate(50)
      Time_Series = df.fetch()
      data.append(Time_Series)
   
    

In [38]:
data = pd.concat(data)

In [39]:
data = data.fillna(0)

In [40]:
data = data.drop(columns=['tsun', 'wpgt', 'snow', 'prcp'])

In [41]:
lat = np.array([stations.loc[station, 'latitude'] for station in true_stations])
long = np.array([stations.loc[station, 'longitude'] for station in true_stations])

In [42]:
param = data.mean(), data.std()

In [44]:
normalize_data = (data - param[0])/param[1]

In [48]:
time = data.index
data = normalize_data.reset_index(level = 'time', drop = True)

In [50]:
time2 = np.unique(time)

In [51]:
param = [data.mean(), data.std()]

In [ ]:
def look_back(data, look, stations) : 
    num_station = len(stations)
    X = [[np.ravel(data.iloc[[i for i in range(k*num_station, (k+1)*num_station)]]) for k in range(j,j+look)] for j in range(int(len(data)/(num_station))-look+1)]
    Y = [X[k][0] for k in range(1,len(X))]
    return np.array(X[:-1]),np.array(Y)

In [ ]:
X,Y = look_back(data, 10, true_stations)

In [95]:
np.save('data', X)
np.save('result', Y)
np.save('time', time2.astype('datetime64[h]'))
np.save('stations', true_stations)
np.save('param', param)
np.save('lat', lat)
np.save('long', long)